<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: Calwebb_coron3 for MIRI coronagraphic imaging
<span style="color:red"> **Instruments Affected**</span>: MIRI, NIRCam

### Table of Contents

<div style="text-align: left"> 
    
<br> [Introduction](#intro)
<br> [JWST CalWG Algorithm](#algorithm)
<br> [Defining Terms](#terms)
<br> [Test Description](#test_descr)
<br> [Data Description](#data_descr)
<br> [Imports](#imports)
<br> [Load Input Data](#data_load)
<br> [Run the Pipeline](#run_pipeline)
<br> [Examine Outputs](#testing) 
<br> [About This Notebook](#about)
<br>    

</div>

<a id="intro"></a>
# Introduction

This is the validation notebook for stage 3 coronagraphic processing of MIRI 4QPM exposures. The stage 3 coronagraphic pipeline ([`calwebb_coron3`](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html#calwebb-coron3)) is to be applied to associations of calibrated NIRCam and MIRI coronagraphic exposures, and is used to produce PSF-subtracted, resampled, combined images of the source object. For more information on `calwebb_coron3`, please visit the links below.

> Module description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html#calwebb-coron3

> Pipeline code: https://github.com/spacetelescope/jwst/blob/master/jwst/coron/


[Top of Page](#title_ID)

<a id="algorithm"></a>
# JWST CalWG Algorithm


The algorithms for `CALWEBB_CORON3` are as follows:

- **Assemble Reference PSFs**: <br>
All the available reference PSFs are assembled into the appropriate association.


- **Outlier detection**: <br> 
An iterative sigma clipping algorithm is used in pixel coordinates on the image stack. The presence of an outlier results in a pixel flag being set.


- **Align reference PSFs**: <br>
The reference PSFs are aligned with the target observation using the Fourier LSQ algorithm to measure the shifts and the Fourier Shift algorithm to apply the shifts to each reference PSF integration.


- **Reference PSF subtraction**: <br>
The reference PSF that is subtracted from each target integration is created using the list of reference PSFs and the KLIP algorithm. 


- **Image Combination**: <br>
The target images (including those at different rotations) are combined into a single combined image using the AstroDrizzle code (with the output pixel size set to the input pixel size).


- **Updated Exposure Level Products**: <br>
The exposure level products are re-created to provide the highest quality products that include the results of the ensemble processing (updated WCS, matching backgrounds, and 2nd pass outlier detection). 

<BR>

The current status of these algorithms are summarized in the link below:

> https://outerspace.stsci.edu/display/JWSTCC/CALWEBB_CORON3


[Top of Page](#title_ID)

<a id="terms"></a>
# Defining Terms

- **JWST**: James Webb Space Telescope ([see documentation](https://jwst-docs.stsci.edu/))
- **MIRI**: Mid-Infrared Instrument ([see documentation](https://jwst-docs.stsci.edu/mid-infrared-instrument))
- **NIRCam**: Near-Infrared Instrument ([see documentation](https://jwst-docs.stsci.edu/near-infrared-camera))
- **4QPM**: 4 Quadrant Phase Mask ([see documentation](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-instrumentation/miri-coronagraphs#MIRICoronagraphs-4qpm))
- **Lyot**: coronagraph design incorporating a classical Lyot spot ([see documentation](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-instrumentation/miri-coronagraphs#MIRICoronagraphs-lyotcoron))
- **PanCAKE**: an in-house tool at STScI used to simulate coronagraphic PSFs 
([see documentation](https://github.com/spacetelescope/pandeia-coronagraphy))
- **SGD**: Small Grid Dither 
([see documentation](https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-high-contrast-imaging/hci-proposal-planning/hci-small-grid-dithers))
    
    

    
 [Top of Page](#title_ID)

<a id="test_descr"></a>
# Test Description

This notebook tests the the following steps applied by `calwebb_coron3` for pipeline version == **'0.17.1'**.

 - [**stack_refs**](#stack_refs)
 - [**align_refs**](#align-refs)
 - [**klip**](#klip)

These tests are performed using simulated MIRI 4QPM coronagraphic data (see [Data Description](#data_descr)).

This notebook does not test the following steps applied by the `calwebb_coron3` pipeline:

 - **outlier_detection**
 - **resample**
 
See [Required Future Testing](#future_tests) for details.


[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description

### Input Data:

The set of data used in these tests were generated using `PanCAKE` and edited to enable processing through the `calwebb_coron3` pipeline. The simulated data was generated for the MIRI 1065C 4QPM coronagraph and consists of one science exposure and nine reference PSF exposures based on the following observation scenario: (1) a science observation of a target star with two faint companions, followed by (2) the execution of a 9-point small grid dither (SGD) pattern on a PSF calibrator of similar magnitude, to obtain a set of 9 slightly offset reference PSF observations.


The data has the following naming format:
- Science exposure: 

      'new_targ_0.fits' 

- Reference PSF exposures:

      'new_ref_0.fits', 'new_ref_1.fits', 'new_ref_2.fits', 'new_ref_3.fits', 'new_ref_4.fits', 'new_ref_5.fits', 'new_ref_6.fits', 'new_ref_7.fits', 'new_ref_8.fits'
      


### Refence Files:

The `align_refs` step requires a PSFMASK reference file containing a 2D mask that’s used as a weight function when computing shifts between images. 

> File description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/align_refs/description.html#psfmask-reffile

Currently the PSFMASK reference files ingested into CRDS are incorrect (wrong shape and incorrectly centered around coronagraphic obstructions), therefore an updated file is used for these tests:

       'psfmask_MIRI_4QPM_1065.fits'
 

### Association File:

Currently the individual stage 3 coronagraphic processing steps can only be run in a convenient way by running the `calwebb_coron3` pipeline on an association (ASN) file that lists the various science target and reference PSF exposures to be processed. 

> Level 3 Associations documentation: https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/level3_asn_rules.html

We use the following ASN file for the purpose of these tests:

       'test.yml'




[Top of Page](#title_ID)

<a id="imports_ID"></a>
# Imports

* `astropy.io` for opening fits files
* `jwst` is the JWST Calibration Pipeline
* `jwst.Coron3Pipeline` is the pipeline being tested
* `matplotlib.pyplot.plt` to generate plots
* `numpy` for array calculations and manipulation
* `download_file` for downloading and accessing files
* `ipywidgets`, `IPython.display.display,clear_output` to display images.
* `ci_watson.arartifactory_helpers.get_bigdata` to download data

[Top of Page](#title_ID)

In [ ]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

In [ ]:
import jwst
from jwst.pipeline import Coron3Pipeline
from astropy.io import fits
from ci_watson.artifactory_helpers import get_bigdata
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.utils.data import download_file
%config InlineBackend.close_figures=False # To prevent automatic figure display when execution of the cell ends
import numpy as np
import ipywidgets as widgets
from IPython.display import display,clear_output
import os

In [ ]:
jwst.__version__
# should out '1.3.2'

<a id="data_load"></a>
# Load Input Data

In [ ]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import shutil
data_dir = TemporaryDirectory()
shutil.copy("jwst_calcoron3_miri_test.yml", os.path.join(data_dir.name, "jwst_calcoron3_miri_test.yml"))
os.chdir(data_dir.name)

Define some plotting helper functions that will overlay the SIAF aperture boundaries and reference point on the data

In [ ]:
# download science image

target_psf_fn = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_targ.fits')

In [ ]:
# download PSF reference images
    
new_ref_0 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_0.fits')

new_ref_1 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_1.fits')

new_ref_2 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_2.fits')

new_ref_3 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_3.fits')

new_ref_4 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_4.fits')

new_ref_5 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_5.fits')

new_ref_6 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_6.fits')

new_ref_7 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_7.fits')
             
new_ref_8 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'calwebb_coron3',
                     'coron3_miri_test', 
                     'new_ref_8.fits')

In [ ]:
# update test data headers with CORONMSK keyword, which is currently missing
img_files = [target_psf_fn,
             new_ref_0, new_ref_1, new_ref_2,
             new_ref_3, new_ref_4, new_ref_5, 
             new_ref_6, new_ref_7, new_ref_8]

for f in img_files:
    with fits.open(f) as hdul:
        hdul[0].header['CORONMSK'] = f'4QPM_{fits.getval(target_psf_fn, "SUBARRAY", 0)[-4:]}'
        hdul.writeto(hdul.filename(), overwrite=True)

In [ ]:
# Create array containing the input reference images
input_ref_images = [fits.getdata(new_ref_8)[0], fits.getdata(new_ref_7)[0], fits.getdata(new_ref_6)[0], 
                    fits.getdata(new_ref_5)[0],fits.getdata(new_ref_4)[0], fits.getdata(new_ref_3)[0], 
                    fits.getdata(new_ref_2)[0], fits.getdata(new_ref_1)[0], fits.getdata(new_ref_0)[0]]

[Top of Page](#title_ID)
<a id="run_pipeline"></a>


------------

# Run the Pipeline

In [ ]:
asn_dir = 'jwst_calcoron3_miri_test.yml'                       # Define ASN file
myCoron3Pipeline = Coron3Pipeline()                              
myCoron3Pipeline.resample.skip = True                          # Skip resample step
myCoron3Pipeline.save_results = True                             
myCoron3Pipeline.output_dir = os.getcwd() 
myCoron3Pipeline.run(asn_dir)                                  # run pipeline

[Top of Page](#title_ID)
<a id="testing"></a>
--------------
# Examine Output Data 

In [ ]:
# First, here are some plotting utitlies to overlay SIAF information about the subarray
# over the images

from pysiaf import Siaf
miri_siaf = Siaf("MIRI")
aper_name = f'MIRIM_{fits.getval(target_psf_fn, "SUBARRAY", 0)}'

def plot_with_aper(img, aper_name=aper_name, ax=None, fig_kws={}, plot_kws={}):
    """
    Plot an image with the aperture outline and reference point overlaid
    img: 2-D image
    aper_name: SIAF-searchable MIRI aperture name. Must match the image footprint.
    ax [None]: axis to draw on. If None, creates new figure
    fig_kws [{}]: figure arguments to pass to plt.subplots()
    plot_kws [{}]: arguments to pass to the plotting function (e.g. ax.imshow(img, **plot_kws))
    Output:
    returns the current fig and ax as a tuple (fig, ax)
    """
    if ax == None:
        fig, ax = plt.subplots(1, 1, **fig_kws)
    else:
        fig = ax.get_figure()
    ax.set_xlabel("subarray pixel")
    ax.set_ylabel("subarray pixel")
    
    aper = Siaf("MIRI")[aper_name]
    aper.plot(frame='sci', ax=ax, fill=False, mark_ref=True, c='C1')

    # using the aperture corner definitions to set the plot coordinates
    # ensures that the plotted image will match up with the aperture's 
    # features, e.g. the reference point
    corners = aper.corners('sci')
    x, y = np.meshgrid(np.arange(corners[0].min(), corners[0].max()+1, 1),
                       np.arange(corners[1].min(), corners[1].max()+1, 1))
    
    ax.pcolor(x, y, img, zorder=-1, **plot_kws)
    
    lolim, hilim = np.min(corners, axis=1), np.max(corners, axis=1)
    ax.set_xlim(lolim[0]-5, hilim[0]+5)
    ax.set_ylim(lolim[1]-5, hilim[1]+5)
    return fig, ax

def plot_stack_with_aper(images, aper_name=aper_name, fig_kws={}, plot_kws={}):
    """
    Similar to plot_with_aper, but you pass a cube in and it makes one plot for each image.
    This function creates its own figure and axes

    images: 3-D cube of images
    aper_name: SIAF-searchable MIRI aperture name. Must match the image footprint.
    fig_kws [{}]: figure arguments to pass to plt.subplots()
    plot_kws [{}]: argumets to pass to the plotting function (e.g. ax.imshow(img, **plot_kws))
    Output:
    returns the created figure
    """
    # initialize the figure
    ncols = 3
    nrows = np.ceil(images.shape[0]/ncols).astype(int)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                             figsize=(6*ncols, 6*nrows),
                             squeeze=False)
    plot_kws = {'vmin':0, 'vmax': 20}
    for index, (ax, img) in enumerate(zip(axes.ravel(), images)):
        plot_with_aper(img, aper_name=aper_name, ax=ax, 
                       fig_kws={}, plot_kws=plot_kws);
    return fig

<a id="stack_refs"></a>
###  `stack_refs`:  Stack PSF References (*'_psfstack' product*)

The role of the `stack_refs` step is to stack all of the PSF reference exposures (specified in the input ASN file) into a single `CubeModel` for use by subsequent coronagraphic steps. The size of the stack should be equal to the sum of the number of integrations in each input PSF exposure.  The image data are simply copied and reformatted and should not be modified in any way.

*Output*: **3D PSF Image Stack** <br>
*File suffix*: **'_psfstack'**

> Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/stack_refs/index.html#stack-refs-step

In [ ]:
stacked_cube_hdu = fits.open('jw10005-miri-mask1065_psfstack.fits')
ref_images = stacked_cube_hdu[1].data
print(stacked_cube_hdu.info())

In [ ]:
print("'_psfstack' data product dimensions: "+str(ref_images.shape))

Show just the stacked images

In [ ]:
plot_kws = {'vmin':0, 'vmax': 20}
fig = plot_stack_with_aper(ref_images, plot_kws=plot_kws)
fig.suptitle("Stacked references");

Show the stacked images, each differenced against the one before it (the first image is shown as normal)

In [ ]:
# subtract each image from the one before it
images = np.diff(ref_images, axis=0, prepend=0)

plot_kws = {'vmin':0, 'vmax': 20}
fig = plot_stack_with_aper(images, plot_kws=plot_kws)
fig.suptitle("Stacked References, Differenced");

The `stack_psfs` step has sucessfully stacked the reference PSF exposures into a single 3D '*_psfstack*' product, with size equal to the sum of the number of integrations in each input PSF exposure *(9)*. To confirm that the image data has not been modified, the input PSF images are subtracted from each image in the stack below:

-----------

<a id="align_refs"></a>


### `align_refs`:  Align PSF References (*'_psfalign' product*)

The role of the `align_refs` step is to align the coronagraphic PSF images with science target images. It does so by computing the offsets between the science target and reference PSF images, and shifts the PSF images into alignment. The output of the `align_refs` step is a 4D data product, where the 3rd axis has length equal to the total number of reference PSF images in the input PSF stack and the 4th axis has length equal to the number of integrations in the input science target product. 

*Output*: **4D aligned PSF Images** <br>
*File suffix*: **_psfalign**


> Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/align_refs/index.html#align-refs-step

In [ ]:
aligned_cube_hdu = fits.open('new_targ_c1001_psfalign.fits')
aligned_cube_hdu.info()

In [ ]:
aligned_cube_data = (aligned_cube_hdu[1].data)
print("'_psfalign' data product dimensions: " + str(aligned_cube_data.shape))

In [ ]:
# cubes after alignment
images = aligned_cube_data[0]

plot_kws = {'vmin':0, 'vmax': 20}
fig = plot_stack_with_aper(images, plot_kws=plot_kws)
fig.suptitle("Aligned References");

In [ ]:
# subtract each aligned image from the one before it
images = np.diff(aligned_cube_data[0], axis=0, prepend=0)
    
plot_kws = {'vmin':0, 'vmax': 20}
fig = plot_stack_with_aper(images, plot_kws=plot_kws)
fig.suptitle("Aligned References, Differenced");

The `align_refs` step has successfully aligned the psf images - note the smaller residuals in the difference images

The output is indeed a 4D '*_psfalign*' product, where the 3rd axis has length equal to the total number of reference images in the input PSF stack *(9)* and 4th axis equal to the number of integrations in the input science target image *(1)*. 

------------
<a id="klip"></a>
### `klip`:  Reference PSF Subtraction

The role of the `klip` step is to apply the Karhunen-Loeve Image Plane (KLIP) algorithm on the science target images, using an accompanying set of aligned reference PSF images (result of the `align_refs` step) in order to fit and subtract an optimal PSF from the science target image. The PSF fitting and subtraction is applied to each integration image independently. The output is a 3D stack of PSF-subtracted images of the science target, having the same dimensions as the input science target product.

*Output*: **3D PSF-subtracted image** <br>
*File suffix*: **_psfsub**

> Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/klip/index.html#klip-step


In [ ]:
sub_hdu = fits.open('new_targ_c1001_psfsub.fits')
sub_hdu.info()

In [ ]:
subtracted_image = sub_hdu[1].data
print("Science target image dimensions: " + str(fits.getdata(target_psf_fn, 'SCI').shape))
print("PSF subtracted image dimensions: " + str(subtracted_image.shape))

Note that the PSF subtracted image has the same dimensions as the input target image.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
fig, ax = plot_with_aper(subtracted_image[0], aper_name=aper_name, ax=ax, plot_kws={'vmin':0,'vmax':10})
ax.set_title("PSF subtracted image")

The `klip` step has successfully made a synthetic psf reference image and subtracted it from the target PSF - indeed, the two companion PSFs that were injected into the target PSF are now visable. The output stack of PSF-subtracted images has the same dimensions as the input science target product.

[Top of Page](#title_ID)
<br>
<a id="future_tests"></a>
--------------
# Required Future Testing

- Testing of the `outlier_detection` step in conjunction with the three steps above.
- Testing of the `resample` step using a data set containing a reference PSF target with astrophysical contamination (i.e. a companion) and target images at two different orientations (simulating referenced differential imaging) - whereby the the `resample` step should correctly combine the two PSF-subtracted target images based on the WCS information. 
- Testing for LYOT, 1140C/FQPM and 1550C/FQPM datasets (using the updated PSFMASK Reference Files provided).
- Testing of multiple integration images (current dataset treated as only single integration images).
- Testing of data that has been processed through stage 1 and 2 pipeline modules.

[Top of Page](#title_ID)

--------------------

<a id="about"></a>
## About this Notebook
**Authors:** 
- Bryony F. Nickson (Staff Scientist, *MIRI Branch*) 
- J. Brendan Hagan
- Jonathan Aguilar (Staff Scientist, *MIRI Branch*)

<br> **Updated On:** 01/12/2022

[Top of Page](#title_ID)